In [113]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [114]:
# To load the environment variable defined in the .env file
from dotenv import load_dotenv
load_dotenv();

In [115]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.calibration import CalibrationDisplay
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc

import src.data.NHLDataManager as DataManager
import src.features.build_features as FeaturesManager

In [118]:
seasons_year = [2015, 2016, 2017, 2018, 2019]
features_data_df = FeaturesManager.build_features(seasons_year)
features_data_df.sample(10)

/mnt/d/GitHub/IFT6758-A2022-G08/src/features/build_features.py:60: RuntimeWarning: invalid value encountered in double_scalars
  


In [66]:
# (goal_events, shot_events, all_events) = data_manager.get_goals_and_shots(2016, "Regular", 429)
# event_idx = 11
# print(all_events[event_idx-1]['about']['periodTime'])
# time_event_s = int(all_events[event_idx-1]['about']['periodTime'].split(':')[0]) * 60 + int(all_events[event_idx-1]['about']['periodTime'].split(':')[1])

01:07


In [74]:
time_event_s

67